In [ ]:
#pip install cloudinary

## Solution 1: Upload jpg file to cloudinary 

Current Approach

In [10]:
import os
import cloudinary.uploader

import numpy as np
import json

In [ ]:
# setup cloudinary

cloudinary.config( 
  cloud_name = "duyoevfl6", 
  api_key = "993237567744246", 
  api_secret = "6o8dniEpExcosXCC1_QFeVb5pTg" 
)

In [ ]:
np.random.seed(10)
num_imgs_upload = 1000
random_numbers = np.random.randint(0, 33502, 10)

In [ ]:
def uploadImage(local_path, id):
    cloudinary.uploader.upload(local_path,
                               folder='DSC106 MET Images',
                               public_id=id)

def get_image_data(root_folder):
    image_paths = []
    path_base = 'https://res.cloudinary.com/duyoevfl6/image/upload/DSC106%20MET%20Images/'
    i = 0
    for subdir, _, files in os.walk(root_folder):
        # only takes the first image file in each subdire folder
        img_file = files[0]
        if img_file.lower().endswith(('.jpg')):  # checking the extension of the files

            image_path = os.path.join(subdir, img_file)
            id = subdir[subdir.rfind('/')+1:]

            uploadImage(image_path, id)
            img_path = {'id': id, 'path': path_base + id + '.jpg'}

            image_paths.append(img_path)

            # only upload 20 images
            i+=1
            if i > 19:
                return image_paths
    return 1

image_directory = 'data/MET'
img_path_data = get_image_data(image_directory)

In [ ]:
# with open('small_img_data.json', 'w') as json_file:
#     json.dump(img_path_data, json_file)

## Solution 2: Convert jpg file to svg file

Does not work as intended -> file size does not decrease

In [23]:
import cv2
import numpy as np
import svgwrite

# def convert_jpg_to_svg(jpg_path, svg_path):
#     # Read the image
#     img = cv2.imread(jpg_path, cv2.IMREAD_GRAYSCALE)
    
#     # Threshold the image to create a binary image
#     _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

#     # Find contours
#     contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#     # Create an SVG drawing
#     dwg = svgwrite.Drawing(svg_path, profile='tiny')

#     # Add contours to the SVG drawing
#     for contour in contours:
#         path_data = "M " + " L ".join(f"{pt[0][0]},{pt[0][1]}" for pt in contour) + " Z"
#         dwg.add(dwg.path(d=path_data, fill='none', stroke='black'))

#     # Save the SVG file
#     dwg.save()

def convert_color_jpg_to_svg(jpg_path, svg_path):
    # Read the image in color
    img = cv2.imread(jpg_path)

    # Convert the image to RGB (OpenCV loads images in BGR by default)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply Gaussian Blur to reduce noise and smooth the image
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)

    # Convert the image to a binary image
    gray = cv2.cvtColor(img_blur, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an SVG drawing
    dwg = svgwrite.Drawing(svg_path, profile='tiny')

    # Add contours to the SVG drawing
    for contour in contours:
        # Create a mask for each contour
        mask = np.zeros_like(img)
        cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

        # Extract the color within the contour from the original image
        color = cv2.mean(img, mask=mask[:,:,0])[:3]  # Get the RGB color

        # Convert the contour to a path string
        path_data = "M " + " L ".join(f"{pt[0][0]},{pt[0][1]}" for pt in contour) + " Z"

        # Add the path to the SVG drawing with the extracted color
        dwg.add(dwg.path(d=path_data, fill=svgwrite.rgb(color[0], color[1], color[2]), stroke='none'))

    # Save the SVG file
    dwg.save()

In [24]:
def get_image_data(root_folder):
    image_paths = []

    i = 0
    for subdir, _, files in os.walk(root_folder):
        # only takes the first image file in each subdire folder
        img_file = files[0]
        if img_file.lower().endswith(('.jpg')):  # checking the extension of the files

            image_path = os.path.join(subdir, img_file)
            
            convert_jpg_to_svg(image_path, 'test.svg')

            # only upload 20 images
            i+=1
            if i > 0:
                return image_paths
    return 1

image_directory = 'data/MET'

# img_path_data = get_image_data(image_directory)

data/MET/390655/1.jpg


In [ ]:
# from PIL import Image
# import svgwrite

# def convert_jpg_to_svg(jpg_path, svg_path):
#     # Open the image and convert it to grayscale
#     image = Image.open(jpg_path).convert('L')
    
#     # Threshold the image to make it binary (black and white)
#     threshold = 128
#     image = image.point(lambda p: p > threshold and 255)
#     image = image.convert('1')
    
#     # Create a bitmap from the image
#     bitmap = potrace.Bitmap(image)
    
#     # Trace the bitmap to a path
#     path = bitmap.trace()
    
#     # Create an SVG drawing
#     dwg = svgwrite.Drawing(svg_path, profile='tiny')
    
#     # Add paths to the SVG drawing
#     for curve in path:
#         start_point = curve.start_point
#         d = f'M {start_point.x} {start_point.y} '
#         for segment in curve:
#             if segment.is_corner:
#                 d += f'L {segment.c.x} {segment.c.y} '
#             else:
#                 d += f'C {segment.c1.x} {segment.c1.y} {segment.c2.x} {segment.c2.y} {segment.end_point.x} {segment.end_point.y} '
#         d += 'Z'
#         dwg.add(dwg.path(d, fill='black'))
    
#     # Save the SVG file
#     dwg.save()

# # Convert the JPG to SVG
# convert_jpg_to_svg('input.jpg', 'output.svg')

In [1]:
import json
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors

In [2]:
with open('data/image_paths.json', 'r') as file:
    path = json.load(file)

In [3]:
np.random.seed(100)
x = np.random.rand(1000)
y = np.random.rand(1000)

small_path = path[:1000]

for idx, path_ in enumerate(small_path):
    path_['org_pos_x'] = round(x[idx], 3)
    path_['org_pos_y'] = round(y[idx], 3)

In [10]:
# with open('small_image_paths.json', 'w') as json_file:
#     json.dump(small_path, json_file)


In [32]:
np.random.seed(4900)
example_size = 15

knn_indicies = np.random.randint(0, len(small_path)-1, size=example_size)
knn_example_path = [small_path[idx] for idx in knn_indicies]

coord = np.array([(node['org_pos_x'], node['org_pos_y']) for node in knn_example_path])
node_ids = [node['id'] for node in knn_example_path]


small_knn_indicies = np.random.choice(len(knn_example_path)-1, size=6, replace=False)
small_knn_example = [knn_example_path[idx] for idx in small_knn_indicies]

small_coord = np.array([(node['org_pos_x'], node['org_pos_y']) for node in small_knn_example])
small_node = [node['id'] for node in small_knn_example]

def knn_to_links(coord, node_ids, k=1):
    neigh = NearestNeighbors(n_neighbors=k+1)  # +1 because a node is also its own neighbor
    neigh.fit(coord)
    distances, indices = neigh.kneighbors(coord)

    links = []
    for i, node_index in enumerate(indices):
        for neighbor_index in node_index[1:]:  # Skip the first neighbor (itself)
            links.append({
                "source": node_ids[i],
                "target": node_ids[neighbor_index]
            })
    
    return links

nodes = [{'id': node['id'], 'name': node['id']} for node in knn_example_path]
small_nodes = [{'id': node['id'], 'name': node['id']} for node in small_knn_example]

knn_example_link = dict()

knn_example_link['nodes_info'] = knn_example_path
knn_example_link['nodes'] = nodes

knn_example_link['small_node_info'] = small_knn_example
knn_example_link['small_node'] = small_nodes
knn_example_link['small_link'] = knn_to_links(small_coord, small_node, k=2)

# create links for knn - k from 1 to 14
for k in range(1, example_size):
    link = knn_to_links(coord, node_ids, k=k)

    knn_example_link[f'link_k_{k}'] = link

knn_example_link

{'nodes_info': [{'id': 3781,
   'path': 'MET/3781/0.jpg',
   'org_pos_x': 0.481,
   'org_pos_y': 0.081},
  {'id': 2341,
   'path': 'MET/2341/0.jpg',
   'org_pos_x': 0.733,
   'org_pos_y': 0.965},
  {'id': 5859,
   'path': 'MET/5859/1.jpg',
   'org_pos_x': 0.021,
   'org_pos_y': 0.481},
  {'id': 2968,
   'path': 'MET/2968/0.jpg',
   'org_pos_x': 0.86,
   'org_pos_y': 0.274},
  {'id': 7217,
   'path': 'MET/7217/2.jpg',
   'org_pos_x': 0.738,
   'org_pos_y': 0.758},
  {'id': 10049,
   'path': 'MET/10049/0.jpg',
   'org_pos_x': 0.726,
   'org_pos_y': 0.051},
  {'id': 3242, 'path': 'MET/3242/0.jpg', 'org_pos_x': 0.239, 'org_pos_y': 0.7},
  {'id': 3480,
   'path': 'MET/3480/0.jpg',
   'org_pos_x': 0.275,
   'org_pos_y': 0.341},
  {'id': 8024,
   'path': 'MET/8024/2.jpg',
   'org_pos_x': 0.048,
   'org_pos_y': 0.964},
  {'id': 7975, 'path': 'MET/7975/2.jpg', 'org_pos_x': 0.31, 'org_pos_y': 0.04},
  {'id': 2218,
   'path': 'MET/2218/3.jpg',
   'org_pos_x': 0.473,
   'org_pos_y': 0.364},
  {'id

In [33]:
knn_example_link['small_node']

[{'id': 2968, 'name': 2968},
 {'id': 3480, 'name': 3480},
 {'id': 7975, 'name': 7975},
 {'id': 10049, 'name': 10049},
 {'id': 3242, 'name': 3242},
 {'id': 2341, 'name': 2341}]

In [34]:
with open('knn_ex_network.json', 'w') as json_file:
    json.dump(knn_example_link, json_file)